In [1]:
library("data.table")
library(samr)
library(plyr)

In [2]:
setwd("/home/veraalva/projects/rna-seq/results/PRJNA339968/differential_expression")

In [3]:
factors = as.data.frame(fread("../../../data/PRJNA339968/PRJNA339968_SraRunTable.tsv"))
head(factors)
nrow(factors)

BioSample,Experiment,Library_Name,LoadDate,MBases,MBytes,Run,SRA_Sample,Sample_Name,isolate,...,LibrarySelection,LibrarySource,Organism,Platform,ReleaseDate,SRA_Study,age,biomaterial_provider,sex,tissue
SAMN05616144,SRX2043366,classical01,2016-08-24,641,491,SRR4053795,SRS1636082,classical01,classical01,...,RT-PCR,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-12-20,SRP082682,Not collected,Kim lab,Not collected,FACS-purified monocytes
SAMN05616145,SRX2043367,classical02,2016-08-24,1623,1196,SRR4053796,SRS1636083,classical02,classical02,...,RT-PCR,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-12-20,SRP082682,Not collected,Kim lab,Not collected,FACS-purified monocytes
SAMN05616154,SRX2043368,classical13,2016-08-24,1405,973,SRR4053797,SRS1636084,classical13,classical13,...,RT-PCR,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-12-20,SRP082682,Not collected,Kim lab,Not collected,FACS-purified monocytes
SAMN05616155,SRX2043369,classical14,2016-08-24,1170,818,SRR4053798,SRS1636085,classical14,classical14,...,RT-PCR,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-12-20,SRP082682,Not collected,Kim lab,Not collected,FACS-purified monocytes
SAMN05616156,SRX2043370,classical15,2016-08-24,666,474,SRR4053799,SRS1636086,classical15,classical15,...,RT-PCR,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-12-20,SRP082682,Not collected,Kim lab,Not collected,FACS-purified monocytes
SAMN05616157,SRX2043371,classical16,2016-08-24,881,641,SRR4053800,SRS1636087,classical16,classical16,...,RT-PCR,TRANSCRIPTOMIC,Homo sapiens,ILLUMINA,2016-12-20,SRP082682,Not collected,Kim lab,Not collected,FACS-purified monocytes


[1] 34

In [4]:
prefix <- "^"
condition1 <- "classical"
condition1exp <- paste(prefix, condition1, sep="")
condition2 <- "nonclassical"
condition2exp <- paste(prefix, condition2, sep="")
condition <- c()
for(i in factors$Sample_Name){
    if (grepl(condition1exp, i)){
        condition <- c(condition,condition1)
    }else if (grepl(condition2exp, i)){
        condition <- c(condition,condition2)
    }
}
factors$Condition <- condition

In [5]:
min_p_value = 0.05
highlight_color = "red"
min_count = 10
min_number_samples = length(condition[condition == 'classical'])

In [6]:
data <- as.data.frame(fread("../quantification/ExonReads.tsv"))
data <- data[,c(c(1,2,3,4,5),match(factors$Run,names(data)))]
nrow(data)
head(data)

[1] 50151

Gene_Id,Chr,Start,End,ExonLength,SRR4053795,SRR4053796,SRR4053797,SRR4053798,SRR4053799,...,SRR4053819,SRR4053820,SRR4053822,SRR4053821,SRR4053823,SRR4053824,SRR4053825,SRR4098781,SRR4098782,SRR4098783
ENSG00000237683.5,chr1,134900,139378,2661,43,4,59,61,22,...,85,34,29,390,3,12,38,100,105,106
ENSG00000241860.2,chr1,141473,173861,6195,4,0,0,33,13,...,5,7,6,15,1,1,37,15,15,10
ENSG00000230021.3,chr1,536815,659929,1697,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000225972.1,chr1,564441,564812,372,769,14,2610,11,8,...,1025,1835,6,2314,456,11,1361,2019,6,929
ENSG00000225630.1,chr1,565019,566062,1044,537,1490,2668,1569,563,...,1069,1283,857,1115,223,270,649,1294,1594,806
ENSG00000237973.1,chr1,566453,567995,1543,1024,1164,125,3023,137,...,3508,1758,2916,2128,2517,965,6273,11805,3226,7501


In [7]:
data.counts <- data[, -c(1:5)]
data.counts[is.na(data.counts)] <- 0
row.names(data.counts) <- data[, 1]
data.counts[] <- as.integer(round(as.matrix(data.counts)))
keep <- (rowSums(data.counts > min_count) >= min_number_samples)
genes_filtered <- rownames(data.counts[keep, ])
data.counts <- data.counts[genes_filtered,]
data <- subset(data, data$Gene_Id %in% genes_filtered)
nrow(data.counts)
head(data.counts)

[1] 9619

,SRR4053795,SRR4053796,SRR4053797,SRR4053798,SRR4053799,SRR4053800,SRR4053801,SRR4053802,SRR4053803,SRR4053804,...,SRR4053819,SRR4053820,SRR4053822,SRR4053821,SRR4053823,SRR4053824,SRR4053825,SRR4098781,SRR4098782,SRR4098783
ENSG00000237683.5,43,4,59,61,22,45,7,4,0,48,...,85,34,29,390,3,12,38,100,105,106
ENSG00000225972.1,769,14,2610,11,8,979,254,136,3,4,...,1025,1835,6,2314,456,11,1361,2019,6,929
ENSG00000225630.1,537,1490,2668,1569,563,552,145,185,175,861,...,1069,1283,857,1115,223,270,649,1294,1594,806
ENSG00000237973.1,1024,1164,125,3023,137,2161,557,363,871,218,...,3508,1758,2916,2128,2517,965,6273,11805,3226,7501
ENSG00000229344.1,21,67,59,19,8,18,2,6,10,18,...,63,26,17,30,4,14,24,24,53,31
ENSG00000248527.1,1040,3171,3122,3058,1510,1258,490,232,1190,1678,...,3023,1879,1745,3805,716,778,1766,3765,2890,2326


In [8]:
groupings <- as.factor(factors$Condition)
samfit <- SAMseq(data.counts, groupings, resp.type = "Two class unpaired", geneid=row.names(data.counts), fdr=0.05)

Estimating sequencing depths...
Resampling to get new data matrices...
perm= 1
perm= 2
perm= 3
perm= 4
perm= 5
perm= 6
perm= 7
perm= 8
perm= 9
perm= 10
perm= 11
perm= 12
perm= 13
perm= 14
perm= 15
perm= 16
perm= 17
perm= 18
perm= 19
perm= 20
perm= 21
perm= 22
perm= 23
perm= 24
perm= 25
perm= 26
perm= 27
perm= 28
perm= 29
perm= 30
perm= 31
perm= 32
perm= 33
perm= 34
perm= 35
perm= 36
perm= 37
perm= 38
perm= 39
perm= 40
perm= 41
perm= 42
perm= 43
perm= 44
perm= 45
perm= 46
perm= 47
perm= 48
perm= 49
perm= 50
perm= 51
perm= 52
perm= 53
perm= 54
perm= 55
perm= 56
perm= 57
perm= 58
perm= 59
perm= 60
perm= 61
perm= 62
perm= 63
perm= 64
perm= 65
perm= 66
perm= 67
perm= 68
perm= 69
perm= 70
perm= 71
perm= 72
perm= 73
perm= 74
perm= 75
perm= 76
perm= 77
perm= 78
perm= 79
perm= 80
perm= 81
perm= 82
perm= 83
perm= 84
perm= 85
perm= 86
perm= 87
perm= 88
perm= 89
perm= 90
perm= 91
perm= 92
perm= 93
perm= 94
perm= 95
perm= 96
perm= 97
perm= 98
perm= 99
perm= 100
Number of thresholds chosen (all poss

In [9]:
allSam<-rbind(samfit$siggenes.table$genes.up,samfit$siggenes.table$genes.lo)
file_name = "samseq.csv"
write.table(allSam, file_name, row.names=F, na="NA", append = F, quote= FALSE, sep = ",", col.names = T)
